In [23]:
import numpy as np
from matplotlib import pyplot as plt
from matplotlib.patches import Rectangle
import torch 
import torch.nn.functional as F
from torch import nn
import pandas as pd
from auto_LiRPA import BoundedModule, BoundedTensor, PerturbationLpNorm

In [24]:
cd /Users/ncohmpli/Documents/Professionnel/Technical/Verification/Lirpa for IoU/manip/pipeline

/Users/ncohmpli/Documents/Professionnel/Technical/Verification/Lirpa for IoU/manip/pipeline


In [25]:
class NeuralNetwork_BrightnessContrast(nn.Module):
    '''
    New model that takes as input a brightness or constrat value and apply it to a specific image
    using the linear_perturbation layer
    (aka the weight and biases of the linear_perturbation layer are set to specific values to encode
    brightness or contrast for a specific image)
    '''
    def __init__(self, classif=True):
        super(NeuralNetwork_BrightnessContrast, self).__init__()
        self.classif=classif
        seed = 0
        torch.manual_seed(seed)
        padding = 1
        self.linear_perturbation = nn.Linear(1,90*90)
        self.conv0 = nn.Conv2d(1, 16, 3, padding= padding) # 3x3 filters w/ same padding
        self.pool0 = nn.MaxPool2d(2, stride=2)
        self.conv1 = nn.Conv2d(16, 16, 3, padding= padding) # 3x3 filters w/ same padding
        self.pool1 = nn.MaxPool2d(2, stride=2)
        self.flatten = nn.Flatten()
        self.linear_relu_stack= nn.Linear(7744,256)
        if self.classif:
            self.linear = nn.Linear(256, 10)
        else:
            self.linear_all = nn.Linear(256, 4) 

    
    def forward(self, alpha):
        layer = self.linear_perturbation(alpha)
        layer = layer.view((-1, 1, 90, 90))
        layer = self.conv0(layer)
        layer = F.relu(self.pool0(layer))
        layer = self.conv1(layer)
        layer = F.relu(self.pool1(layer))
        layer = self.flatten(layer)
        layer = self.linear_relu_stack(layer)
        layer = F.relu(layer)
        if self.classif:
            logits = self.linear(layer)
        else:
            logits = self.linear_all(layer)
        return logits

In [26]:
model_test = NeuralNetwork_BrightnessContrast(classif=False)
model_test(torch.Tensor([0.001]))

tensor([[-0.1079, -0.0068, -0.0729, -0.0156]], grad_fn=<AddmmBackward0>)

## Digit model

In [27]:
model_digit = NeuralNetwork_BrightnessContrast(classif=True)

filename = 'toy_model_classif'
model_torch_load  = torch.jit.load(f'{filename}.pt')
digit_config = model_torch_load.state_dict()
digit_config['linear_perturbation.weight']=torch.Tensor(np.zeros((90*90, 1), dtype='float32'))
digit_config['linear_perturbation.bias']=torch.Tensor(np.zeros((90*90,), dtype='float32'))
model_digit.load_state_dict(digit_config)
#model_torch_load.state_dict()

<All keys matched successfully>

## Corner Model

In [28]:
model_corners = NeuralNetwork_BrightnessContrast(classif=False)

filename = 'toy_model_corners'
model_torch_load  = torch.jit.load(f'{filename}.pt')
corner_config = model_torch_load.state_dict()
# we create values for the linear_perturbation parameters that will me set for brightness and contrast afterwards
corner_config['linear_perturbation.weight']=torch.Tensor(np.zeros((90*90, 1), dtype='float32'))
corner_config['linear_perturbation.bias']=torch.Tensor(np.zeros((90*90,), dtype='float32'))
model_corners.load_state_dict(corner_config)
#model_torch_load.state_dict()


<All keys matched successfully>

## Load Data

In [29]:
cd utils

/Users/ncohmpli/Documents/Professionnel/Technical/Verification/Lirpa for IoU/manip/pipeline/utils


In [30]:
ls

__pycache__/   custom_op.py   data/          model_func.py  train.csv


In [31]:
from model_func import *
from torch.utils.data import DataLoader

train_df = pd.read_csv("train.csv")

In [32]:
dict_df_10 = {}
for label in np.unique(list(train_df["label"])): 
    print(label)
    dict_df_10[label] = train_df[train_df["label"] == label].head(10)

0
1
2
3
4
5
6
7
8
9


In [33]:
train_df_100 = pd.concat(dict_df_10.values())
trainingData = CustomMnistDataset_OL(train_df_100)
train_dataloader = DataLoader(trainingData, batch_size=1, shuffle=False)
iterator = iter(train_dataloader)

In [34]:
data = next(iter(train_dataloader))
X_0, _ = data
data = next(iter(train_dataloader))
X_1, _ = data

In [35]:
# create brightness values
brightness_range = np.asarray(np.linspace(0, 0.5, 100), 'float32')
brightness_min = brightness_range[:-1]
brightness_max = brightness_range[1:]

brightness_center = (brightness_min+brightness_max)/2.
eps_brightness = ((brightness_max-brightness_min)/2.).max()

X_brightness = torch.Tensor(brightness_center[:,None])

## Brightness

brightness(x; alpha)= x+ alpha

thus for linear_perturbation

linear_perturbation.weight = [1..1]

linear_perturbation.weight = x

## Contrast

contrast(x; alpha)= x*alpha

thus for linear_perturbation

linear_perturbation.weight = x

linear_perturbation.bias = [0..0]

In [36]:
def set_brightness(model, image):
    # create a pytorch model for a specific flatten image
    # once a model has been set for one images, you can call it for any brightness value
    image_flatten = image.view((90*90,))
    brightness_config = model.state_dict()
    brightness_config['linear_perturbation.bias']=image_flatten
    brightness_config['linear_perturbation.weight']=torch.Tensor(np.ones((90*90,1), dtype='float32'))
    model.load_state_dict(brightness_config)
    

## AutoLirpa on brightness

In [44]:
model_lirpa_digit = BoundedModule(model_digit, torch.tensor([0.0]))


In [49]:
ptb_brightness = PerturbationLpNorm(norm=np.inf, eps=X_brightness[0].item()) #attention espilon c'est X_brightness


In [53]:
input_lirpa_brightness = BoundedTensor(torch.tensor([0.0]), ptb_brightness)
input_lirpa_brightness

<BoundedTensor: BoundedTensor([0.]), PerturbationLpNorm(norm=inf, eps=0.0025252525229007006)>

In [55]:
model_lirpa_digit(torch.tensor([0.0]))

tensor([[-10.0793,  16.3645,  -1.0624,  -6.1694,  -2.2533,  -0.0239,  -1.3484,
          -1.0524,  -4.4216,  -1.7593]], grad_fn=<AddBackward0>)

In [56]:
model_lirpa_digit.compute_bounds(x=(input_lirpa_brightness,),IBP=True, method='crown')

be careful Noémie is doing shit on auto lirpa [BoundInput(name="/0"), BoundParams(name="/1"), BoundParams(name="/2"), BoundParams(name="/3"), BoundParams(name="/4"), BoundParams(name="/5"), BoundParams(name="/6"), BoundParams(name="/7"), BoundParams(name="/8"), BoundParams(name="/9"), BoundParams(name="/10")] ['/0', '/1', '/2', '/3', '/4', '/5', '/6', '/7', '/8', '/9', '/10']
<BoundedTensor: BoundedTensor([0.]), PerturbationLpNorm(norm=inf, eps=0.0025252525229007006)>
BoundInput(name="/0")
be careful added by Noemie
be careful added by Noemie


RuntimeError: shape '[10, 1]' is invalid for input of size 81000

In [ ]:
ptb_brightness = PerturbationLpNorm(norm=np.inf, eps=X_brightness[0].item() #attention espilon c'est X_brightness
input_lirpa_brightness = BoundedTensor(torch.tensor([0.0]), ptb_brightness)

lb_brightness, ub_brightness = model_lirpa_digit.compute_bounds(x=(input_lirpa_brightness,),
                                                    IBP=True, method='crown')

In [47]:
def expe_brightness(X_0, model_digit): 
    set_brightness(model_digit, X_0)
    dico_brightness = {}
    for brightness_variations in X_brightness:
        model_lirpa_digit = BoundedModule(model_digit, torch.tensor([0.0]))
        ptb_brightness = PerturbationLpNorm(norm=np.inf, eps=brightness_variations.item()) #attention espilon c'est X_brightness
        input_lirpa_brightness = BoundedTensor(torch.tensor([0.0]), ptb_brightness)
        
        lb_brightness, ub_brightness = model_lirpa_digit.compute_bounds(x=(input_lirpa_brightness,),
                                                            IBP=True, method='crown')
        dico_brightness[brightness_variations] = (lb_brightness, ub_brightness)
    return(dico_brightness)

dico_brightness_X0 = expe_brightness(X_0, model_corners)

be careful Noémie is doing shit on auto lirpa [BoundInput(name="/0"), BoundParams(name="/1"), BoundParams(name="/2"), BoundParams(name="/3"), BoundParams(name="/4"), BoundParams(name="/5"), BoundParams(name="/6"), BoundParams(name="/7"), BoundParams(name="/8"), BoundParams(name="/9"), BoundParams(name="/10")] ['/0', '/1', '/2', '/3', '/4', '/5', '/6', '/7', '/8', '/9', '/10']
<BoundedTensor: BoundedTensor([0.]), PerturbationLpNorm(norm=inf, eps=0.0025252525229007006)>
BoundInput(name="/0")
be careful added by Noemie
be careful added by Noemie


RuntimeError: shape '[4, 1]' is invalid for input of size 32400

In [42]:
dico_brightness_X0

NameError: name 'dico_brightness_X0' is not defined

In [21]:
train_df_100[0:3]

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
dico = {}
for image_id in range(len(train_df_100[)):
    list_info = []
    print(f"Begin to work with image {image_id}")
    data = next(iterator)
    X, y = data
    lb, ub = expe_brightness(X, model_digit)
    dico[image_id] = lb, ub

Begin to work with image 0
be careful Noémie is doing shit on auto lirpa [BoundInput(name="/0"), BoundParams(name="/1"), BoundParams(name="/2"), BoundParams(name="/3"), BoundParams(name="/4"), BoundParams(name="/5"), BoundParams(name="/6"), BoundParams(name="/7"), BoundParams(name="/8"), BoundParams(name="/9"), BoundParams(name="/10")] ['/0', '/1', '/2', '/3', '/4', '/5', '/6', '/7', '/8', '/9', '/10']
<BoundedTensor: BoundedTensor([[0.0025],
               [0.0076],
               [0.0126],
               [0.0177],
               [0.0227],
               [0.0278],
               [0.0328],
               [0.0379],
               [0.0429],
               [0.0480],
               [0.0530],
               [0.0581],
               [0.0631],
               [0.0682],
               [0.0732],
               [0.0783],
               [0.0833],
               [0.0884],
               [0.0934],
               [0.0985],
               [0.1035],
               [0.1086],
               [0.1136],
   

In [ ]:
dico